# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\desai\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable
from newsapi.newsapi_client import NewsApiClient
load_dotenv()
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])
print(type(newsapi))

<class 'newsapi.newsapi_client.NewsApiClient'>


In [3]:
mapbox_key = os.getenv('mapbox')
print(type(mapbox_key))


<class 'str'>


In [4]:
# Fetch the Bitcoin news articles
bitcoin_news = newsapi.get_everything(q='bitcoin', language='en')

In [5]:
# Fetch the Ethereum news articles
ethereum_news = newsapi.get_everything(q='ethereum', language='en')

In [6]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_sentiment = []

for article in bitcoin_news['articles']:
    try:
        text = article['content']
        date = article['publishedAt'][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment['compound']
        pos = sentiment['pos']
        neu = sentiment['neu']
        neg = sentiment['neg']

        bitcoin_sentiment.append({
            'text': text,
            'date': date,
            'compound': compound,
            'positive': pos,
            'negative': neg,
            'neutral': neu
        })

    except AttributeError:
        pass

#creating dataframe
bitcoin_df = pd.DataFrame(bitcoin_sentiment)
bitcoin_df.head()

,text,date,compound,positive,negative,neutral
0,Even if you arent paying attention to Bitcoin ...,2022-02-09,0.5859,0.124,0.000,0.876
1,When Denis Rusinovich set up cryptocurrency mi...,2022-01-12,0.0000,0.000,0.000,1.000
2,El Salvador introduced Bitcoin as a legal tend...,2022-01-25,0.3182,0.105,0.000,0.895
3,Were officially building an open Bitcoin minin...,2022-01-14,-0.4404,0.000,0.083,0.917
4,Israeli national Tal Prihar pled guilty to rou...,2022-01-27,-0.3182,0.045,0.084,0.871


In [7]:
# Create the Ethereum sentiment scores DataFrame
ethereum_sentiment = []

for article in ethereum_news['articles']:
    try:
        text = article['content']
        date = article['publishedAt'][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment['compound']
        pos = sentiment['pos']
        neu = sentiment['neu']
        neg = sentiment['neg']

        ethereum_sentiment.append({
            'text': text,
            'date': date,
            'compound': compound,
            'positive': pos,
            'negative': neg,
            'neutral': neu
        })

    except AttributeError:
        pass

#creating dataframe
ethereum_df = pd.DataFrame(ethereum_sentiment)
ethereum_df.head()


,text,date,compound,positive,negative,neutral
0,"In a new blog post the company said that 4,836...",2022-01-20,0.0000,0.000,0.000,1.000
1,Hackers who made off with roughly $15 million ...,2022-01-19,0.0000,0.000,0.000,1.000
2,"On some level, the new mayor is simply employi...",2022-01-20,0.1779,0.052,0.000,0.948
3,"Back in September\r\n, Robinhood announced pla...",2022-01-21,0.0772,0.038,0.000,0.962
4,If people who buy cryptocurrencies intended on...,2022-02-10,-0.2023,0.039,0.062,0.899


In [8]:
# Describe the Bitcoin Sentiment
bitcoin_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.039480,0.060950,0.041250,0.897800
std,0.393121,0.059169,0.049184,0.068563
min,-0.493900,0.000000,0.000000,0.729000
25%,-0.344025,0.000000,0.000000,0.874750
50%,0.038600,0.054000,0.000000,0.912000
75%,0.328950,0.108750,0.085000,0.934000
max,0.680800,0.185000,0.151000,1.000000


In [9]:
# Describe the Ethereum Sentiment
ethereum_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.133000,0.064100,0.022100,0.913800
std,0.331538,0.064006,0.045603,0.078963
min,-0.680800,0.000000,0.000000,0.775000
25%,0.000000,0.000000,0.000000,0.875500
50%,0.038600,0.048500,0.000000,0.913000
75%,0.406800,0.106000,0.011750,1.000000
max,0.757900,0.217000,0.174000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin

Q: Which coin had the highest compound score?

A: Ethreum

Q. Which coin had the highest positive score?

A: Ethreum

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [10]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [11]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()
# Create a list of stopwords
sw = stopwords.words('english')
# Expand the default stopwords list if necessary


In [12]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    regex = re.compile('[^a-zA-Z]')
    re_clean = regex.sub(' ', text)
    # Create a tokenized list of the words
    re_words = word_tokenize(re_clean)
    # Lemmatize words into root words
    tokens = [lemmatizer.lemmatize(word) for word in re_words]
    # Convert the words to lowercase and remove stop words
    re_words = [word.lower() for word in re_words if word.lower() not in sw]    
    
    return tokens

In [13]:
# Create a new tokens column for Bitcoin
bitcoin_df['tokens'] = bitcoin_df['text']
bitcoin_df.head()

,text,date,compound,positive,negative,neutral,tokens
0,Even if you arent paying attention to Bitcoin ...,2022-02-09,0.5859,0.124,0.000,0.876,Even if you arent paying attention to Bitcoin ...
1,When Denis Rusinovich set up cryptocurrency mi...,2022-01-12,0.0000,0.000,0.000,1.000,When Denis Rusinovich set up cryptocurrency mi...
2,El Salvador introduced Bitcoin as a legal tend...,2022-01-25,0.3182,0.105,0.000,0.895,El Salvador introduced Bitcoin as a legal tend...
3,Were officially building an open Bitcoin minin...,2022-01-14,-0.4404,0.000,0.083,0.917,Were officially building an open Bitcoin minin...
4,Israeli national Tal Prihar pled guilty to rou...,2022-01-27,-0.3182,0.045,0.084,0.871,Israeli national Tal Prihar pled guilty to rou...


In [14]:
# Create a new tokens column for Ethereum
ethereum_df['tokens'] = ethereum_df['text']
ethereum_df.head()

,text,date,compound,positive,negative,neutral,tokens
0,"In a new blog post the company said that 4,836...",2022-01-20,0.0000,0.000,0.000,1.000,"In a new blog post the company said that 4,836..."
1,Hackers who made off with roughly $15 million ...,2022-01-19,0.0000,0.000,0.000,1.000,Hackers who made off with roughly $15 million ...
2,"On some level, the new mayor is simply employi...",2022-01-20,0.1779,0.052,0.000,0.948,"On some level, the new mayor is simply employi..."
3,"Back in September\r\n, Robinhood announced pla...",2022-01-21,0.0772,0.038,0.000,0.962,"Back in September\r\n, Robinhood announced pla..."
4,If people who buy cryptocurrencies intended on...,2022-02-10,-0.2023,0.039,0.062,0.899,If people who buy cryptocurrencies intended on...


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [15]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
processed = tokenizer(bitcoin_df.iloc[0]['text'])
bitcoin_counts = Counter(ngrams(processed, n=2))
print(dict(bitcoin_counts))

{('Even', 'if'): 1, ('if', 'you'): 1, ('you', 'arent'): 1, ('arent', 'paying'): 1, ('paying', 'attention'): 1, ('attention', 'to'): 1, ('to', 'Bitcoin'): 1, ('Bitcoin', 'and'): 1, ('and', 'other'): 1, ('other', 'cryptocurrencies'): 1, ('cryptocurrencies', 'you'): 1, ('you', 'might'): 1, ('might', 'have'): 1, ('have', 'noticed'): 1, ('noticed', 'that'): 1, ('that', 'their'): 1, ('their', 'value'): 1, ('value', 'plummeted'): 1, ('plummeted', 'last'): 1, ('last', 'week'): 1, ('week', 'with'): 1, ('with', 'the'): 1, ('the', 'total'): 1, ('total', 'value'): 1, ('value', 'of'): 1, ('of', 'the'): 1, ('the', 'market'): 1, ('market', 'tumbling'): 1, ('tumbling', 'from'): 1, ('from', 'a'): 1, ('a', 'high'): 1, ('high', 'of'): 1, ('of', 'tr'): 1, ('tr', 'char'): 1}


In [17]:
# Generate the Ethereum N-grams where N=2
processed_2 = tokenizer(ethereum_df.iloc[0]['text'])
ethereum_counts = Counter(ngrams(processed_2, n=2))
print(dict(ethereum_counts))

{('In', 'a'): 2, ('a', 'new'): 1, ('new', 'blog'): 1, ('blog', 'post'): 2, ('post', 'the'): 1, ('the', 'company'): 1, ('company', 'said'): 1, ('said', 'that'): 1, ('that', 'ETH'): 1, ('ETH', 'and'): 1, ('and', 'bitcoin'): 1, ('bitcoin', 'were'): 1, ('were', 'taken'): 1, ('taken', 'Illustration'): 1, ('Illustration', 'by'): 1, ('by', 'Alex'): 1, ('Alex', 'Castro'): 1, ('Castro', 'The'): 1, ('The', 'Verge'): 1, ('Verge', 'In'): 1, ('a', 'blog'): 1, ('post', 'published'): 1, ('published', 'in'): 1, ('in', 'the'): 1, ('the', 'early'): 1, ('early', 'hour'): 1, ('hour', 'of'): 1, ('of', 'Thursday'): 1, ('Thursday', 'morning'): 1, ('morning', 'cryptocurrency'): 1, ('cryptocurrency', 'char'): 1}


In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
token_count(bitcoin_counts, N=10)

[(('Even', 'if'), 1),
 (('if', 'you'), 1),
 (('you', 'arent'), 1),
 (('arent', 'paying'), 1),
 (('paying', 'attention'), 1),
 (('attention', 'to'), 1),
 (('to', 'Bitcoin'), 1),
 (('Bitcoin', 'and'), 1),
 (('and', 'other'), 1),
 (('other', 'cryptocurrencies'), 1)]

In [20]:
# Use token_count to get the top 10 words for Ethereum
token_count(ethereum_counts, N=10)

[(('In', 'a'), 2),
 (('blog', 'post'), 2),
 (('a', 'new'), 1),
 (('new', 'blog'), 1),
 (('post', 'the'), 1),
 (('the', 'company'), 1),
 (('company', 'said'), 1),
 (('said', 'that'), 1),
 (('that', 'ETH'), 1),
 (('ETH', 'and'), 1)]

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
big_string = ' '.join(bitcoin_counts)
wc = WordCloud().generate(big_string)
plt.imshow(wc)

TypeError: sequence item 0: expected str instance, tuple found

In [23]:
# Generate the Ethereum word cloud
big_string_2 = ' '.join(ethereum_counts)
wc_2 = WordCloud().generate(big_string_2)
plt.imshow(wc_2)

TypeError: sequence item 0: expected str instance, tuple found

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
bitcoin_text = ' '.join(bitcoin_df['text'])
bitcoin_text

'Even if you arent paying attention to Bitcoin and other cryptocurrencies, you might have noticed that their value plummeted last week, with the total value of the market tumbling from a high of $3 tr… [+4782 chars] When Denis Rusinovich set up cryptocurrency mining company Maveric Group in Kazakhstan in 2017, he thought he had hit the jackpot. Next door to China and Russia, the country had everything a Bitcoin … [+4140 chars] El Salvador introduced Bitcoin as a legal tender alongside the US dollar in 2021. | Illustration by Alex Castro / The Verge\r\n\n \n\n The International Monetary Fund’s executive board has recommended El … [+2691 chars] Were officially building an open Bitcoin mining system\r\nPhoto by Joe Raedle/Getty Images\r\nBlock, the payment company formerly known as Square, is working on building an open Bitcoin mining system, it… [+2078 chars] Israeli national Tal Prihar pled guilty to routing money from kickbacks through Bitcoin and shell companies \r\nIllustration by Al

In [28]:
# Run the NER processor on all of the text
bitcoin_doc = nlp(bitcoin_text)
# Add a title to the document
bitcoin_doc.user_data['title'] = 'Bitcoin NER'

In [29]:
# Render the visualization
displacy.render(bitcoin_doc, style='ent')

In [30]:
# List all Entities
for ent in bitcoin_doc.ents:
    print(ent.text, ent.label_)

last week DATE
3 MONEY
Denis Rusinovich PERSON
Maveric Group ORG
Kazakhstan GPE
2017 DATE
China GPE
Russia GPE
El Salvador GPE
US GPE
2021 DATE
Alex Castro PERSON
Verge ORG
The International Monetary Fund’s ORG
El ORG
Bitcoin PERSON
Joe Raedle/Getty Images PERSON
Square FAC
Bitcoin PERSON
Israeli NORP
Tal Prihar PERSON
Illustration ORG
Alex Castro PERSON
Verge ORG
two-year DATE
4,836 CARDINAL
ETH ORG
443 CARDINAL
Alex Castro PERSON
Verge ORG
the early hours TIME
Thursday DATE
morning TIME
Saturday DATE
more than 50% PERCENT
November DATE
more than $ MONEY
Jack Dorsey PERSON
Twitter PRODUCT
first ORDINAL
October last year DATE
the "Giga Texas ORG
Elon Musk PERSON
Friday DATE
Fintech ORG
New York City GPE
British NORP
Richard ODell PERSON
house ORG
October DATE
Poulde GPE
Russia GPE
Thursday DATE
Russian NORP
the last five years DATE
about 25,000 CARDINAL
Lichtensteins PERSON
Bitcoin PERSON
El Salvador GPE
US GPE
The International Monetary Fund ORG
El Salvador GPE
Changpeng PERSON
Binanc

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
ethereum_text = ' '.join(ethereum_df['text'])
ethereum_text

'In a new blog post the company said that 4,836 ETH and 443 bitcoin were taken\r\nIllustration by Alex Castro / The Verge\r\nIn a blog post published in the early hours of Thursday morning, cryptocurrency… [+2004 chars] Hackers who made off with roughly $15 million in ethereum from Crypto.com are attempting to launder the funds through a so-called ethereum mixer, known as Tornado Cash, according to a new report from… [+3891 chars] On some level, the new mayor is simply employing an age-old political strategy: making overtures to the business community. Fintech is a growing field in New York City, and blockchain-based companies… [+1562 chars] Back in September\r\n, Robinhood announced plans to test a cryptocurrency wallet within its app. At the time, the company said it would open the beta to a small number of people before expanding availa… [+1069 chars] If people who buy cryptocurrencies intended only to hold on to them as speculative investments, there\'d be no real need for crypto w

In [32]:
# Run the NER processor on all of the text
ethereum_doc = nlp(ethereum_text)
# Add a title to the document
ethereum_doc.user_data['title'] = 'Ethereum NER'

In [33]:
# Render the visualization
displacy.render(ethereum_doc, style='ent')

In [34]:
# List all Entities
for ent in ethereum_doc.ents:
    print(ent.text, ent.label_)

4,836 CARDINAL
ETH ORG
443 CARDINAL
Alex Castro PERSON
Verge ORG
the early hours TIME
Thursday DATE
morning TIME
roughly $15 million MONEY
Crypto.com ORG
Tornado Cash PERSON
Fintech ORG
New York City GPE
September DATE
Robinhood ORG
Crypto.com ORG
about $34 million MONEY
Monday DATE
Saul Alinsky Ethereum ORG
Ethereum ORG
Ethereum GPE
418% PERCENT
2021 DATE
AMC Entertainment ORG
AMC ORG
GME ORG
Ethereums ORG
more than 650% PERCENT
Vitalik Buterin PERSON
Ethereum ORG
Feb 7 DATE
Reuters ORG
Monday DATE
$450 million MONEY
Sequoia Capital India ORG
OPEN ORG
second ORDINAL
OPEN ORG
second ORDINAL
nearly $324 million MONEY
Getty Images PERSON
more than $324 million MONEY
Wormhole PERSON
120,000 CARDINAL
wEth ORG
Ethereum ORG
nearly $324 million MONEY
Getty Images PERSON
more than $324 million MONEY
Wormhole PERSON
Bloomberg TV ORG
Crypto.com ORG
Kris Marszalek PERSON
400 CARDINAL
British NORP
Richard ODell PERSON
house ORG
October DATE
Poulde GPE
Feb 4 DATE
Reuters ORG
8.82% PERCENT
40,611.4 

---